# Problem 3 Naive Bayes

## 3.1 implement a Naive Bayes classifier

In [27]:
import sys
import math
import itertools
from collections import Counter
import numpy as np
import pandas as pd
from matplotlib import pyplot as plt
import time
from copy import deepcopy

In [93]:
class Model:
    def __init__(self, wordlist):
        self.wordlist = wordlist

    def count_labels(self, data):
        """
        Count the number of positive labels and negative labels.
        Returns (a tuple or a numpy array of two elements):
            * negative_count: a non-negative integer, which represents the number of negative labels (non-spam emails);
            * positive_count: a non-negative integer, which represents the number of positive labels (spam emails).
        """
        # TODO
        result = np.zeros((2))
        pos = 0
        neg = 0
        for i in range(len(data)):
            if data[i][0] == 1:
                pos += 1
            else:
                neg += 1
                
        result[0] = neg
        result[1] = pos
        return result
    
    def count_words(self, wordlist, data):
        """
        Count the number of times that each word appears in emails under a given label.
        Returns (a numpy array):
            * word_counts: a numpy array with shape (2, L), where L is the length of $wordlist,
                - word_counts[0, i] represents the number of times that word $wordlist[i] appears in non-spam (negative) emails, and
                - word_counts[1, i] represents the number of times that word $wordlist[i] appears in spam (positive) emails.
        """
        # TODO
        L = len(wordlist)
        word_counts = np.zeros((2,L))
#         for i in range(L):
#             for j in data:
#                 if j[0] == 0:
#                     #non spam
#                     for word in j[1]:
#                         if word == wordlist[i]:
#                             word_counts[0][i] += 1
#                             break
#                 else:
#                     #spam
#                     for word in j[1]:
#                         if word == wordlist[i]:
#                             word_counts[1][i] += 1
#                             break
        #transform data into ndarray
        data = np.asarray(data)
        print("shape of data: ",data.shape)
        spam_data = np.hstack(data[np.where(data[:,0]==1)][:,1])
#         print("check spam data: ",spam_data)
        nons_data = np.hstack(data[np.where(data[:,0]==0)][:,1])
        
        
        print("shape of spam data: ",spam_data.shape)
        
        values, counts = np.unique(spam_data, return_counts=True)
        spam_dict = dict(zip(values,counts))
        
        values, counts = np.unique(nons_data, return_counts=True)
        nons_dict = dict(zip(values,counts))
        
        for i in range(L):
            word = wordlist[i]
            if word in spam_dict:
                word_counts[1][i] = spam_dict[word]
                print(word,spam_dict[word])
            else:
                word_counts[1][i] = 0
            
            if word in nons_dict:
                word_counts[0][i] = nons_dict[word]
                print(word,nons_dict[word])
            else:
                word_counts[0][i] = 0
        
#         print("word_counts: ",word_counts)
        return word_counts

    def calculate_probability(self, label_counts, word_counts):
        """
        Calculate the probabilities, both the prior and likelihood.
        Returns (a pair of numpy array):
            * prior_probs: a numpy array with shape (2, ), only two elements, where
                - prior_probs[0] is the prior probability of negative labels, and
                - prior_probs[1] is the prior probability of positive labels.
            * likelihood_probs: a numpy array with shape (2, L), where L is the length of the word list,
                - likelihood_probs[0, i] represents the likelihood probability of the $i-th word in the word list, given that the email is non-spam (negative), and
                - likelihood_probs[1, i] represents the likelihood probability of the $i-th word in the word list, given that the email is spam (positive).
        """
        # TODO
        L = word_counts.shape[1]
        prior_probs = np.zeros((2))
        likelihood_probs = np.zeros((2,L))
        prior_probs[0] = label_counts[0] / (label_counts[0]+ label_counts[1])
        prior_probs[1] = label_counts[1] / (label_counts[0]+ label_counts[1])                       
        
        for i in range(word_counts.shape[1]):
            likelihood_probs[0][i] = (word_counts[0][i]+1)/(label_counts[0]+2)
            likelihood_probs[1][i] = (word_counts[1][i]+1)/(label_counts[1]+2)
            
        return prior_probs,likelihood_probs
        
    def fit(self, data):
        label_counts = self.count_labels(data)
        word_counts = self.count_words(self.wordlist, data)
        
        self.label_counts = label_counts
        self.word_counts = word_counts
        self.prior_probs, self.likelihood_probs = self.calculate_probability(label_counts, word_counts)

        # TO AVOID NUMBER OVERFLOW here we use log probability instead.
        self.log_prior_probs = np.log(self.prior_probs)
        self.log_likelihood_probs = np.log(self.likelihood_probs)
#         np.dstack([np.log(1 - self.likelihood_probs), np.log(self.likelihood_probs)])
        
    def predict(self, x):
        """
        Predict whether email $x is a spam or not.
        Returns:
            * y: a boolean value indicating whether $x is a spam or not.
        """        
        p_x_spam = 0
        p_x_non = 0
        
        for word in self.wordlist:
            index = self.wordlist.index(word)
            if word in x:
                p_x_spam += self.log_likelihood_probs[1][index]
                p_x_non += self.log_likelihood_probs[0][index]
            else:
                try:
                    p_x_spam += math.log((1-self.likelihood_probs[1][index]))
                    p_x_non += math.log((1-self.likelihood_probs[0][index]))
                except:
                    print("math error, the prob is ",self.likelihood_probs[1][index], "and",(self.likelihood_probs[0][index]))
                    print("p_x_spam is ",)
                    break
        
        p_x_spam *= self.log_prior_probs[1]
        p_x_non *= self.log_prior_probs[0]
        if p_x_spam >= p_x_non:
            return True
        else:
            return False

In [94]:
def read_data(filename):
    """
    Read the dataset from the file given by name $filename.
    The returned object should be a list of pairs of data. In each pair: the first element is 1 (for spam emails) 
    or 0 (for non-spam emails), the second element is a list of words in the email.
    The returned list: 
        [
            (1 , ['a', 'b', 'c']),
            (0, ['d', 'e', 'f']),
            ...
        ]
    """
    
    result = []
    
    df = pd.read_csv(filename,sep='\n',header=None,names=["email"])
    df = pd.DataFrame(df.email.str.split(' ',1).tolist(),
                                 columns = ['label','text']) 
    for row in df.itertuples(index=False):
        label = int(row.label)
        text = row.text.split(' ')
        new_data = (label,text)
        result.append(new_data)
        
    return result

    
    
def split_train(original_train_data, size=4000):
    return original_train_data[:size], original_train_data[size:]


def create_wordlist(original_train_data, threshold=26):
    """
    Create a word list from the original training set.
    Only get a word if it appears in at least $threshold emails.
    Returns:
        * a python list containing all the words that occur in at least $threshold emails.
    """
    dic = {}
    
    for i in range(len(original_train_data)):
        texts = original_train_data[i][1]
#         print(texts)
        texts = set(texts)
        for word in texts:
            if word in dic:
                dic[word]+=1
            else:
                dic[word] =1
        
    wordlist = []
    for i in dic.keys():
        if dic[i] >= threshold:
            wordlist.append(i)
    
    return wordlist

In [95]:
# threshold to determine whether to include a word in the dictionary/wordlist.
# ie. only words with frequency higher than threshold are included
THRESHOLD = 26

In [96]:
original_train_data = read_data('spam_train.txt')


# further split the data into a training set and a validation set
train_data, val_data = split_train(original_train_data)
print("test")
# Create the word list.
wordlist = create_wordlist(original_train_data, 26)
print("Total # of words:", len(wordlist))

# fit the model using train_data
model = Model(wordlist)
model.fit(train_data)
# print("shape of log: ",model.log_likelihood_probs.shape)

# TODO
# calculate the error rate on val_data (when threshold=26)
# print out the error rate
result = {"spam":0,"non-spam":0}
error_count =0
for i in range(len(val_data)):
    is_spam = model.predict(val_data[i][1])
    if is_spam == True:
        if val_data[i][0] == 0:
            error_count  += 1
    else:
        if val_data[i][0] == 1:
            error_count  += 1

error_percentage = (error_count / len(val_data))*100
print("Validation error, # = {:>4d}, % = {:>8.4f}%.".format(error_count, error_percentage))

test
Total # of words: 3048
shape of data:  (4000, 2)
shape of spam data:  (510362,)
click 1388
click 351
or 2784
or 3190
panel 37
panel 55
can 1529
can 2565
dollarnumb 2658
dollarnumb 619
i 2976
i 9575
name 894
name 464
have 2206
have 3632
e 1400
e 910
promot 162
promot 66
extens 30
extens 60
much 323
much 600
sincer 89
sincer 13
here 1392
here 933
in 5274
in 9772
remov 1138
remov 298
avail 368
avail 378
the 13735
the 31030
current 235
current 350
ar 2589
ar 3425
were 149
were 809
origin 129
origin 392
registr 38
registr 15
be 3087
be 4208
and 9420
and 13869
todai 459
todai 355
easi 321
easi 243
control 110
control 274
at 1406
at 3037
administr 46
administr 154
benefit 169
benefit 89
fee 132
fee 42
discount 71
discount 25
number 29406
number 40905
excit 80
excit 24
well 237
well 665
will 2350
will 1956
regist 153
regist 144
approv 118
approv 50
address 1165
address 538
to 12781
to 18874
you 8768
you 5791
com 481
com 891
for 5384
for 7435
manag 252
manag 491
it 2995
it 9108
from 2139
f

label 14
label 41
safeti 36
safeti 19
terrorist 6
terrorist 84
involv 110
involv 118
id 74
id 385
secur 322
secur 581
offic 128
offic 245
safe 111
safe 57
di 37
di 84
meant 10
meant 39
head 52
head 163
campaign 43
campaign 48
agreement 29
agreement 40
seven 56
seven 24
document 132
document 219
belong 16
belong 20
mind 95
mind 195
box 146
box 253
notifi 13
notifi 22
transact 195
transact 41
william 11
william 88
ownership 18
ownership 20
dear 148
dear 47
arrang 31
arrang 24
understand 100
understand 190
reach 132
reach 100
senior 30
senior 41
famili 269
famili 170
wait 134
wait 133
affili 61
affili 13
mutual 19
mutual 17
arrest 15
arrest 33
conclud 23
conclud 25
urgent 59
urgent 12
reader 69
reader 88
drug 76
drug 31
histori 67
histori 106
wouldn 37
wouldn 84
abl 140
abl 213
proven 76
proven 22
clear 46
clear 147
lift 15
lift 32
hettinga 91
abil 56
abil 114
trade 243
trade 185
poverti 3
poverti 52
anybodi 41
anybodi 47
actual 110
actual 416
connect 70
connect 237
self 65
self 85
seat 2

ear 11
ear 16
overnight 19
overnight 5
director 29
director 90
whom 18
whom 32
role 9
role 58
black 127
black 63
parent 25
parent 68
ne 31
ne 7
layer 3
layer 38
depress 22
depress 21
children 48
children 55
saw 18
saw 61
stage 6
stage 61
evolv 2
evolv 44
numberdnumb 357
numberdnumb 15
settlement 39
settlement 14
bond 14
bond 59
promis 44
promis 77
usa 94
usa 86
she 62
she 285
struggl 11
struggl 29
entertain 30
entertain 53
januari 27
januari 36
grab 12
grab 37
premier 22
premier 20
arrai 5
arrai 94
publish 98
publish 157
ran 2
ran 63
girl 40
girl 61
contest 6
contest 21
pair 1
pair 44
blue 14
blue 64
london 10
london 42
interview 18
interview 59
kid 31
kid 77
previous 34
previous 57
film 23
film 125
photo 21
photo 99
feet 17
feet 53
justin 3
justin 92
femal 39
femal 42
ti 8
ti 80
weird 32
ltd 18
ltd 10
portion 23
portion 37
modul 1
modul 130
skip 20
skip 79
classifi 39
classifi 38
prefix 67
done 65
done 246
hurt 1
hurt 34
gave 15
gave 55
context 39
organ 63
organ 118
guido 36
clue 1
cl

theme 4
theme 45
influenc 8
influenc 27
fell 1
fell 26
profess 21
profess 6
patrick 2
patrick 33
fuel 13
fuel 15
approxim 28
approxim 22
frank 5
frank 25
martin 13
martin 29
transport 5
transport 31
restaur 7
restaur 12
hotel 32
hotel 16
redistribut 3
redistribut 36
xerox 17
xerox 9
divers 4
divers 20
newspap 18
newspap 27
song 5
song 51
weapon 20
weapon 51
helvetica 33
helvetica 18
nationwid 22
nationwid 6
arial 100
arial 20
bargain 13
bargain 8
invit 36
invit 24
anthoni 3
anthoni 34
professor 3
professor 24
acquisit 40
acquisit 14
math error, the prob is  1.1050119331742243 and 0.12813978886057517
the word is  click its occurence is [ 351. 1388.] label:  [2745. 1255.]
math error, the prob is  1.1050119331742243 and 0.12813978886057517
the word is  or its occurence is [3190. 2784.] label:  [2745. 1255.]
math error, the prob is  2.2155926809864757 and 1.1616308700400437
the word is  panel its occurence is [55. 37.] label:  [2745. 1255.]
math error, the prob is  1.1050119331742243 and 0

the word is  below its occurence is [118. 528.] label:  [2745. 1255.]
math error, the prob is  1.1050119331742243 and 0.12813978886057517
the word is  fill its occurence is [ 49. 282.] label:  [2745. 1255.]
math error, the prob is  1.1050119331742243 and 0.12813978886057517
the word is  dai its occurence is [519. 679.] label:  [2745. 1255.]
math error, the prob is  2.2155926809864757 and 1.1616308700400437
the word is  return its occurence is [207. 177.] label:  [2745. 1255.]
math error, the prob is  1.2171837708830548 and 0.9341099381143065
the word is  jersei its occurence is [ 4. 26.] label:  [2745. 1255.]
math error, the prob is  1.1050119331742243 and 0.12813978886057517
the word is  protect its occurence is [208. 174.] label:  [2745. 1255.]
math error, the prob is  1.1050119331742243 and 0.12813978886057517
the word is  kept its occurence is [34. 42.] label:  [2745. 1255.]
math error, the prob is  1.1050119331742243 and 0.12813978886057517
the word is  help its occurence is [537.

the word is  georg its occurence is [62. 12.] label:  [2745. 1255.]
math error, the prob is  1.1050119331742243 and 0.12813978886057517
the word is  base its occurence is [477. 244.] label:  [2745. 1255.]
math error, the prob is  1.1050119331742243 and 0.12813978886057517
the word is  invoic its occurence is [ 4. 17.] label:  [2745. 1255.]
math error, the prob is  1.2171837708830548 and 0.9341099381143065
the word is  million its occurence is [275. 511.] label:  [2745. 1255.]
math error, the prob is  1.1050119331742243 and 0.12813978886057517
the word is  chairman its occurence is [40.  4.] label:  [2745. 1255.]
math error, the prob is  1.1050119331742243 and 0.12813978886057517
the word is  oper its occurence is [263. 116.] label:  [2745. 1255.]
math error, the prob is  1.1050119331742243 and 0.12813978886057517
the word is  although its occurence is [133.  30.] label:  [2745. 1255.]
math error, the prob is  1.1050119331742243 and 0.12813978886057517
the word is  hundr its occurence i

the word is  subscrib its occurence is [225. 170.] label:  [2745. 1255.]
math error, the prob is  1.1050119331742243 and 0.12813978886057517
the word is  competit its occurence is [61. 61.] label:  [2745. 1255.]
math error, the prob is  1.1050119331742243 and 0.12813978886057517
the word is  succe its occurence is [17. 37.] label:  [2745. 1255.]
math error, the prob is  1.1050119331742243 and 0.12813978886057517
the word is  u its occurence is [251. 296.] label:  [2745. 1255.]
math error, the prob is  1.1050119331742243 and 0.12813978886057517
the word is  coast its occurence is [21. 17.] label:  [2745. 1255.]
math error, the prob is  1.1050119331742243 and 0.12813978886057517
the word is  heard its occurence is [101.  43.] label:  [2745. 1255.]
math error, the prob is  1.1050119331742243 and 0.12813978886057517
the word is  postal its occurence is [ 1. 47.] label:  [2745. 1255.]
math error, the prob is  1.1050119331742243 and 0.12813978886057517
the word is  analyst its occurence is [

In [98]:
1-math.log(381/1256)

2.1928879719014427

## 3.2 try different thresholds, find the optimal threshold (which gives minimum validation error), print out the test error at the optimal threshold

In [6]:
def compute_error_rate(model, data):
    
    error_count = sum([y != model.predict(x) for y, x in data])
    return 100.0 * error_count / len(data)

In [7]:
thresholds = list(range(1, 35))
train_error = []
val_error = []
test_error = []
original_train_data = read_data('spam_train.txt')
train_data, val_data = split_train(original_train_data)
test_data = read_data('spam_test.txt')

In [ ]:
for th in thresholds[::-1]:
    print('With threshold {}....'.format(th))
    time1 = time.time()

    # vocabulary selection
    wordlist = create_wordlist(original_train_data, th)

    # fit model using the wordlist
    model = Model(wordlist)
    model.fit(train_data)
    
    print("finish training data ...")
    # compute classification error rates
    err_train = compute_error_rate(model, train_data)
    err_val = compute_error_rate(model, val_data)
    err_test = compute_error_rate(model, test_data)

    # store results for plotting
    train_error.append(err_train)
    val_error.append(err_val)
    test_error.append(err_test)

    time2 = time.time()
    print("train:{} val:{} test:{} len(V)={}".format(err_train, err_val, err_test, len(wordlist)))
    print('time: {}'.format(time2 - time1))

With threshold 34....
finish training data ...
train:5.225 val:6.1 test:6.8 len(V)=3451
time: 1197.6894969940186
With threshold 33....


In [ ]:
# plot the training and validation error rate vs. the thresholds
# choose the threshold with the minimal validation error rate and report the corresponding test error rate

# TODO
plt.subplot(2, 1, 1)
plt.plot(thresholds,train_error[::-1])
plt.xlabel("threshold")
plt.ylabel("train_error")
plt.title("train_error vs threshold")
plt.subplot(2, 1, 2)
plt.plot(thresholds,val_error[::-1],label="val error vs. threshold")
plt.xlabel("threshold")
plt.ylabel("val_error")
plt.title("val_error vs threshold")
plt.legend()
plt.show()

opt = thresholds[val_error.index(min(val_error))]

# print('Best performance at validated threshold {} with test error rate {}.'.format(opt, test_error[opt]))